<a href="https://colab.research.google.com/github/kamakiri-8/gemini/blob/main/GeminiAPI%26Streamlit%26ngrock%E3%81%A7%E3%82%A2%E3%83%97%E3%83%AA%E4%BD%9C%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# パッケージのインストール
!pip install -q -U google-generativeai
!pip install pyngrok
!pip install streamlit

# パッケージのimport
import time
import google.generativeai as genai
import pathlib
import textwrap
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

# 環境変数の準備（左端の鍵アイコンでGOOGLE_API_KEYを設定）
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

# Markdown出力ヘルパーの準備
def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

# モデルの準備
model = genai.GenerativeModel("models/gemini-exp-1121")

# Gemini API使ってファイルをgoogle cloud strageにアップロードしURIを取得する関数
def upload_to_gemini(path, mime_type=None):
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

# APIがファイルを正常に受信したことを確認する
def wait_for_files_active(file_video):
  print("Waiting for file processing...")
  name = file_video.name
  while file_video.state.name == "PROCESSING":
    print(".", end="", flush=True)
    time.sleep(10)
    file_video = genai.get_file(name)
  if file_video.state.name != "ACTIVE":
    raise Exception(f"File {file_video.name} failed to process")
  print("...all files ready")

In [4]:
# Gemini からのレスポンスを取得する処理
response = model.generate_content(
    """
    九州地方の県名を列挙してください。
    # 出力形式
    表
    # 参考フォーマット
    件名　ふりがな
    """
)

# 応答を出力
to_markdown(response.text)

> はい、九州地方の県名を以下の表にまとめました。
> 
> | 県名   | ふりがな   |
> | :----- | :--------- |
> | 福岡県   | ふくおか   |
> | 佐賀県   | さが       |
> | 長崎県   | ながさき   |
> | 熊本県   | くまもと   |
> | 大分県   | おおいた   |
> | 宮崎県   | みやざき   |
> | 鹿児島県 | かごしま   |
> | 沖縄県   | おきなわ   |


In [5]:
import streamlit as st

st.title("Gemini API アプリ")

# プロンプトの入力
prompt = st.text_input("プロンプトを入力してください:")

# ファイルアップローダー
uploaded_file = st.file_uploader("ファイルを選択", type=["txt", "pdf", "csv"])

# Gemini APIをトリガーするボタン
if st.button("送信"):
  if uploaded_file is not None:
    # ファイルアップロードの処理 (upload_to_gemini関数を使用)
    file_uri = upload_to_gemini(uploaded_file)
    # ファイルURIをプロンプトに組み込む
    prompt = f"{prompt} {file_uri}"

  # プロンプトをGemini APIに送信
  response = model.generate_content(prompt)
  # 応答を表示
  st.write(response.text)

2024-11-25 07:06:59.952 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-25 07:07:00.148 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-25 07:07:00.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-25 07:07:00.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-25 07:07:00.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-25 07:07:00.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-25 07:07:00.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-25 07:07:00.162 Session state does not 

In [30]:
# Streamlitの再起動
!pkill -f streamlit

# 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行
!streamlit run app.py &>/dev/null&
!ngrok authtoken "2ohfTYi1JZqBedsDISGksL9L6hy_7V27osRSm3MfPyufnAN5c"
from pyngrok import ngrok

ngrok.kill()  # プロセスの修了
url = ngrok.connect(addr="8501")  # 接続port=8501
print(url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://e01d-34-28-178-146.ngrok-free.app" -> "http://localhost:8501"
